Importing libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.layers import Input,Conv2D, Conv3D, MaxPooling2D, UpSampling2D, MaxPooling3D
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor

Importing the dataset

In [4]:
# Load the dataset
image_folder = r"C:\Users\91897\Desktop\CP302\Speckle Images\C_1\Set1\Center"
csv_file = r"C:\Users\91897\Desktop\CP302\Center Results\Mean_speckle_contrast_values_Center.csv"

# Loading Y(Mean Speckle Contrast) values from CSV
df = pd.read_csv(csv_file)
Y_values = df.iloc[1:92, 4].values.astype(float) 
Y = [f"{value:.4f}" for value in Y_values]
print('Mean Speckle Contrast values [Y]: ', Y_values)
print('No of Y sample:',len(Y))

Mean Speckle Contrast values [Y]:  [0.49653985 0.49189961 0.50423411 0.50089421 0.53147321 0.47378567
 0.48474404 0.46919811 0.49585846 0.51376004 0.56128645 0.50406953
 0.48120438 0.52413103 0.48311468 0.5119267  0.49106547 0.55786441
 0.50545507 0.48657495 0.56362458 0.61200477 0.56078304 0.52123219
 0.53651615 0.56650276 0.60724196 0.5531455  0.54475131 0.51818902
 0.60898229 0.52019796 0.48637313 0.4962693  0.52592649 0.49118076
 0.5052272  0.45886105 0.54163262 0.4742058  0.50209682 0.52235343
 0.58768073 0.52022994 0.50167967 0.48740485 0.45142465 0.50373894
 0.4742886  0.50700819 0.46776733 0.49658549 0.48067961 0.45280526
 0.6208639  0.54935633 0.61174565 0.5022284  0.44011182 0.49306561
 0.46657515 0.50401608 0.52037687 0.47985948 0.48763926 0.48206674
 0.50976678 0.48562817 0.50162886 0.46627197 0.4785888  0.44651616
 0.43401569 0.44743276 0.46985529 0.45357697 0.4894797  0.58991034
 0.53956998 0.4885937  0.4721378  0.46731734 0.49100701 0.51618157
 0.44585688 0.50667365 0.46

In [5]:
# Prepare input data X
X_values = []
for image_file in sorted(os.listdir(image_folder)):
    if image_file.endswith(".png"):
        image_path = os.path.join(image_folder, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Check if the image was successfully loaded
        if img is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        # Resize the image
        img_resized = cv2.resize(img, (629, 836))
        X_values.append(img_resized)

# Convert X & Y list to numpy array and reshape
X_values = np.array(X_values).reshape(-1, 836, 629, 1)
Y = Y_values.astype('float32')

# Normalize pixel values
X_values = X_values.astype('float32') / 255.0  

# Splitting data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_values, Y, test_size=0.2, random_state=42)

# Reshape y_train and y_test to have a single output column
y_train = y_train.reshape(-1, 1).astype('float32')
y_test = y_test.reshape(-1, 1).astype('float32')


CAE model

In [31]:
def create_cae():
    input_img = Input(shape=(836, 629, 1))

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Regression output
    flatten = Flatten()(encoded)
    regression_output = Dense(1, activation='linear')(flatten)

    # Full model
    cae_model = tf.keras.Model(inputs=input_img, outputs=[decoded, regression_output])
    cae_model.compile(optimizer='adam', loss=['mse', 'mse'], metrics=['mae'])
    
    return cae_model


In [32]:
# Create an instance of the CAE model
cae_model = create_cae()

# Display the model summary
cae_model.summary()

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = cae_model.fit(X_train, [X_train, y_train], 
                        epochs=50, 
                        batch_size=8, 
                        validation_data=(X_test, [X_test, y_test]), 
                        callbacks=[early_stopping])


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 836, 629,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 836, 629,  │        320 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 418, 315,  │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 418, 315,  │      4,624 │ max_pooling2d_6[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 209, 158,  │          0 │ conv2d_16[0][0]   │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 209, 158,  │      2,320 │ max_pooling2d_7[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_6     │ (None, 418, 316,  │          0 │ conv2d_17[0][0]   │
│ (UpSampling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 418, 316,  │      4,640 │ up_sampling2d_6[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_7     │ (None, 836, 632,  │          0 │ conv2d_18[0][0]   │
│ (UpSampling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 528352)    │          0 │ max_pooling2d_7[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 836, 632,  │        289 │ up_sampling2d_7[… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │    528,353 │ flatten_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 540,546 (2.06 MB)

 Trainable params: 540,546 (2.06 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


ValueError: Dimensions must be equal, but are 629 and 632 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, functional_3_1/conv2d_19_1/Sigmoid)' with input shapes: [8,836,629,1], [8,836,632,1].